# Imports

In [1]:
import os
import sys
import pandas as pd
import datetime
import subprocess
import os
import json


In [2]:
src_dir = os.path.join(os.getcwd(), '..', 'src')
sys.path.append(src_dir)

In [3]:
from traffic_analysis.d00_utils.data_retrieval import delete_and_recreate_dir
from traffic_analysis.d00_utils.load_confs import load_parameters, load_paths
from traffic_analysis.d02_ref.load_video_names_from_s3 import load_video_names_from_s3
from traffic_analysis.d02_ref.retrieve_and_upload_video_names_to_s3 import retrieve_and_upload_video_names_to_s3
from traffic_analysis.d02_ref.upload_annotation_names_to_s3 import upload_annotation_names_to_s3
from traffic_analysis.d03_processing.update_s3_processed import update_frame_level_table

params = load_parameters()
paths = load_paths()

Using TensorFlow backend.


# Analysis

In [5]:
#retrieve_and_upload_video_names_to_s3(ouput_file_name='test_search',
#                                      paths=paths,
#                                      from_date='2019-06-30',
#                                      to_date='2019-06-30',
#                                      from_time='13-00-00',
#                                      to_time='13-05-00',
#                                      camera_list=params['tims_camera_list'][:2])

From: 2019-06-30 To: 2019-06-30
Extracting 293343 file names for date 2019-06-30 took 110.28195691108704 seconds


In [6]:
#upload_annotation_names_to_s3(paths)

Extracting 24 file names took 0.6831274032592773 seconds


In [4]:
selected_videos = load_video_names_from_s3(ref_file='test_search',
                                           paths=paths)

In [5]:
chunk_size = params['chunk_size']

In [16]:
chunk_size

5

In [ ]:
selected_videos

In [17]:
update_frame_level_table(file_names=selected_videos[6:10],
                         paths=paths,
                         params=params)

Classifying video 0/4.
Classifying video 1/4.
Classifying video 2/4.
Classifying video 3/4.


In [7]:
from traffic_analysis.d03_processing.update_s3_processed import load_s3_parquet

In [8]:
df = load_s3_parquet(file='frame_table', paths=paths)
df.head()

,camera_id,frame_id,date,time,obj_bounds,obj_classification,confidence,video_id
obj_ind,,,,,,,,
0,00001.06590,0,2019-06-29,13:00:40.071379,"[178, 204, 196, 224]",car,0.415502,0
0,00001.06590,1,2019-06-29,13:00:40.071379,"[176, 204, 196, 224]",car,0.437907,0
0,00001.06590,2,2019-06-29,13:00:40.071379,"[181, 204, 199, 224]",car,0.422005,0
0,00001.06590,3,2019-06-29,13:00:40.071379,"[182, 206, 200, 224]",car,0.484743,0
0,00001.06590,4,2019-06-29,13:00:40.071379,"[182, 205, 200, 223]",car,0.557349,0


In [14]:
df.loc[(df.camera_id == '00001.04336') & (df.frame_id == 0)]

,camera_id,frame_id,date,time,obj_bounds,obj_classification,confidence,video_id
obj_ind,,,,,,,,
0,00001.04336,0,2019-06-29,13:01:40.228706,"[118, 138, 138, 156]",car,0.700746,3
1,00001.04336,0,2019-06-29,13:01:40.228706,"[232, 215, 246, 239]",person,0.610739,3
2,00001.04336,0,2019-06-29,13:01:40.228706,"[66, 106, 84, 120]",car,0.467177,3
3,00001.04336,0,2019-06-29,13:01:40.228706,"[120, 101, 130, 109]",car,0.421970,3
4,00001.04336,0,2019-06-29,13:01:40.228706,"[73, 72, 93, 92]",car,0.411990,3


In [15]:
len(df.camera_id.unique())

4

In [4]:
from sqlalchemy import create_engine
db_host = parameters['DATABASE_PARAMS']['db_host']
db_name = parameters['DATABASE_PARAMS']['db_name']
db_user = credentials['dev_sql']['username']
db_pass = credentials['dev_sql']['password']

conn = create_engine('postgresql://%s:%s@%s/%s' % 
                       (db_user, db_pass, db_host, db_name),
                       encoding = 'latin1', 
                       echo = True)

In [9]:
df.to_sql(name='frame_level',con=conn)

2019-07-17 12:04:23,369 INFO sqlalchemy.engine.base.Engine select version()


I0717 12:04:23.369070 140377874048832 log.py:110] select version()


2019-07-17 12:04:23,370 INFO sqlalchemy.engine.base.Engine {}


I0717 12:04:23.370263 140377874048832 log.py:110] {}


2019-07-17 12:04:23,374 INFO sqlalchemy.engine.base.Engine select current_schema()


...

2019-07-17 12:04:23,467 INFO sqlalchemy.engine.base.Engine ({'obj_ind': 0, 'camera_id': '00001.06590', 'frame_id': 0, 'date': datetime.date(2019, 6, 29), 'time': datetime.time(13, 0, 40, 71379), 'obj_bounds': array([178, 204, 196, 224]), 'obj_classification': 'car', 'confidence': 0.41550174355506897, 'video_id': 0}, {'obj_ind': 0, 'camera_id': '00001.06590', 'frame_id': 1, 'date': datetime.date(2019, 6, 29), 'time': datetime.time(13, 0, 40, 71379), 'obj_bounds': array([176, 204, 196, 224]), 'obj_classification': 'car', 'confidence': 0.4379068613052368, 'video_id': 0}, {'obj_ind': 0, 'camera_id': '00001.06590', 'frame_id': 2, 'date': datetime.date(2019, 6, 29), 'time': datetime.time(13, 0, 40, 71379), 'obj_bounds': array([181, 204, 199, 224]), 'obj_classification': 'car', 'confidence': 0.42200514674186707, 'video_id': 0}, {'obj_ind': 0, 'camera_id': '00001.06590', 'frame_id': 3, 'date': datetime.date(2019, 6, 29), 'time': datetime.time(13, 0, 40, 71379), 'obj_bounds': array([182, 206, 2

I0717 12:04:23.467120 140377874048832 log.py:110] ({'obj_ind': 0, 'camera_id': '00001.06590', 'frame_id': 0, 'date': datetime.date(2019, 6, 29), 'time': datetime.time(13, 0, 40, 71379), 'obj_bounds': array([178, 204, 196, 224]), 'obj_classification': 'car', 'confidence': 0.41550174355506897, 'video_id': 0}, {'obj_ind': 0, 'camera_id': '00001.06590', 'frame_id': 1, 'date': datetime.date(2019, 6, 29), 'time': datetime.time(13, 0, 40, 71379), 'obj_bounds': array([176, 204, 196, 224]), 'obj_classification': 'car', 'confidence': 0.4379068613052368, 'video_id': 0}, {'obj_ind': 0, 'camera_id': '00001.06590', 'frame_id': 2, 'date': datetime.date(2019, 6, 29), 'time': datetime.time(13, 0, 40, 71379), 'obj_bounds': array([181, 204, 199, 224]), 'obj_classification': 'car', 'confidence': 0.42200514674186707, 'video_id': 0}, {'obj_ind': 0, 'camera_id': '00001.06590', 'frame_id': 3, 'date': datetime.date(2019, 6, 29), 'time': datetime.time(13, 0, 40, 71379), 'obj_bounds': array([182, 206, 200, 224])

2019-07-17 12:04:23,470 INFO sqlalchemy.engine.base.Engine ROLLBACK


I0717 12:04:23.470086 140377874048832 log.py:110] ROLLBACK


ProgrammingError: (psycopg2.ProgrammingError) can't adapt type 'numpy.ndarray'
[SQL: INSERT INTO frame_level (obj_ind, camera_id, frame_id, date, time, obj_bounds, obj_classification, confidence, video_id) VALUES (%(obj_ind)s, %(camera_id)s, %(frame_id)s, %(date)s, %(time)s, %(obj_bounds)s, %(obj_classification)s, %(confidence)s, %(video_id)s)]
[parameters: ({'obj_ind': 0, 'camera_id': '00001.06590', 'frame_id': 0, 'date': datetime.date(2019, 6, 29), 'time': datetime.time(13, 0, 40, 71379), 'obj_bounds': array([178, 204, 196, 224]), 'obj_classification': 'car', 'confidence': 0.41550174355506897, 'video_id': 0}, {'obj_ind': 0, 'camera_id': '00001.06590', 'frame_id': 1, 'date': datetime.date(2019, 6, 29), 'time': datetime.time(13, 0, 40, 71379), 'obj_bounds': array([176, 204, 196, 224]), 'obj_classification': 'car', 'confidence': 0.4379068613052368, 'video_id': 0}, {'obj_ind': 0, 'camera_id': '00001.06590', 'frame_id': 2, 'date': datetime.date(2019, 6, 29), 'time': datetime.time(13, 0, 4

In [ ]:
query = """
select *
from cases
limit 10;
"""

df = pd.read_sql(query, conn)
df.head()

In [9]:
import os
import yaml

with open(src_dir + '/../conf/local/credentials.yml') as f:
    credentials = yaml.safe_load(f)


In [10]:
print("{}/{}".format(_bucket_name, load_key))

NameError: name '_bucket_name' is not defined

In [11]:
from s3fs.core import S3FileSystem
_credentials = credentials
_s3 = S3FileSystem(client_kwargs=_credentials['dev_s3'])
_bucket_name = 's3://' + paths['bucket_name']
load_key = 'frame_level/frame_table/'

with _s3.open("{}/{}".format(_bucket_name, load_key), mode="rb") as s3_file:
    df = pd.read_parquet(s3_file)

FileNotFoundError: air-pollution-uk/frame_level/frame_table